Models Processing

In [1]:
model_paths = {
        'model1': "cardiffnlp/twitter-roberta-base-sentiment-latest",
        'model2': "nlptown/bert-base-multilingual-uncased-sentiment",
        'model3': "cardiffnlp/twitter-xlm-roberta-base-sentiment",
        'model4': "siebert/sentiment-roberta-large-english",
        'model5': "lxyuan/distilbert-base-multilingual-cased-sentiments-student",
        'model6': "finiteautomata/bertweet-base-sentiment-analysis",
        'model7': "j-hartmann/sentiment-roberta-large-english-3-classes"
}

In [2]:
from transformers import pipeline

models = {name : pipeline('sentiment-analysis', model=model_path) for name, model_path in model_paths.items()}

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device s

Text Importing

In [4]:
import pandas as pd
df = pd.read_excel('posts_first_targil.xlsx', sheet_name=None)

In [5]:
df['J-P'].rename(columns={'Body': 'Body Text'}, inplace=True)

In [6]:
for sheet_name, sheet_df in df.items():
    print(f"Sheet name: {sheet_name}")
    print(sheet_df.columns)

Sheet name: A-J
Index(['sub_title', 'date', 'Newspaper', 'Body Text', 'title'], dtype='object')
Sheet name: BBC
Index(['date', 'Newspaper', 'Body Text', 'title'], dtype='object')
Sheet name: J-P
Index(['date', 'Newspaper', 'Body Text', 'title'], dtype='object')
Sheet name: NY-T
Index(['date', 'Newspaper', 'Body Text', 'title'], dtype='object')


In [7]:
with open('israel.txt', 'r') as israel_file, open('palestine.txt', 'r') as palestine_file:
    i_word = israel_file.read().splitlines()
    p_word = palestine_file.read().splitlines()


In [8]:
import numpy as np
import pandas as pd

csv1 = []

for sheet_name, sheet_df in df.items():
    for index, row in sheet_df.iterrows():

        article_sentences = []

        if pd.notna(row['Body Text']):
            article_sentences += row['Body Text'].split('.')

        if pd.notna(row['title']):
            article_sentences += row['title'].split('.')

        if sheet_name == 'A-J' and pd.notna(row['sub_title']):
            article_sentences += row['sub_title'].split('.')

        for sentence in article_sentences:
            if any(word in sentence for word in i_word) and not any(word in sentence for word in p_word):
                csv1.append({'paper': sheet_name, 'article_index': index, 'sentence': sentence, 'I/P': 'I'})
            elif any(word in sentence for word in p_word) and not any(word in sentence for word in i_word):
                csv1.append({'paper': sheet_name, 'article_index': index, 'sentence': sentence, 'I/P': 'P'})

In [10]:
from tqdm import tqdm

negative_labels = ['NEGATIVE', 'negative', '1 star', '2 stars', 'NEG']
positive_labels = ['POSITIVE', 'positive', '4 stars', '5 stars', 'POS']

for sentence in tqdm(csv1, colour='green'):
    pos, neg, nut, sum_pos, sum_neg, sum_nut = 0, 0, 0, 0, 0, 0
    for model_name, model in models.items():
       res = model(sentence['sentence'])[0]
       sentence[f'{model_name} score'] = res['score']
       sentence[f'{model_name} label'] = 'POS' if res['label'] in positive_labels else 'NEG' if res['label'] in negative_labels else 'NUT'
       pos, neg, nut = pos + (1 if res['label'] == 'POS' else 0), neg + (1 if res['label'] == 'NEG' else 0), nut + (1 if res['label'] == 'NUT' else 0)
       sum_pos, sum_neg, sum_nut = sum_pos + res['score'] if res['label'] == 'POS' else sum_pos, sum_neg + res['score'] if res['label'] == 'NEG' else sum_neg, sum_nut + res['score'] if res['label'] == 'NUT' else sum_nut

    sentence['majority'] = 'POS' if pos > neg and pos > nut else 'NEG' if neg > pos and neg > nut else 'NUT' if nut > pos and nut > neg else 'N/A'
    sentence['AVG-majority-score'] = sum_pos / pos if sentence['majority'] == 'POS' else sum_neg / neg if sentence['majority'] == 'NEG' else sum_nut / nut if sentence['majority'] == 'NUT' else 'N/A'

  1%|          | 41/6043 [00:55<2:15:57,  1.36s/it]


KeyboardInterrupt: 